* Student name: WILLIAM OMONDI NDONI
* Student pace:  full time 

## Business Understanding
WON Real Estate Limited is a real estate company located in the American Northwest that buys and sells houses to its clients. The company would like to develop a model that can estimate the price of a house based on its features. In addition,the company would like to know which features best estimate the house prices.

This project will help identify the features that can be used to provide a good estimate of the house prices.The project will also develop a model that can estimate the house prices and evaluate its performance.

## Data understanding
The data used in this project was obtained from [kaggle website](https://www.kaggle.com/datasets/shivachandel/kc-house-data). This dataset contains the prices of houses in the American Northwest and their various features such as,the number of bathrooms and the number of bedrooms.These are contained in the various columns of the dataset.

The dataset contains twenty one columns that are the features(characteristics) of the houses.The main aim of this project is to estimate the prices of houses that are contained in the 'price' column.
This dataset will therefore be of use since the company is also in the Northwest.

## Data Preparation

### Importing the relevant libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline
sns.set()

#### Loading and exploring the dataset

In [24]:
df = pd.read_csv('Data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
## Getting information about the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

 From the documentation provided on columns we can clearly identify columns such as waterfront,grade,condition and view as categorical.We will further investigate to identify whether there are other categorical columns

In [4]:
## Taking a look at the  sqft_basement column which is seen to be an object data type 
df['sqft_basement'].head()

0      0.0
1    400.0
2      0.0
3    910.0
4      0.0
Name: sqft_basement, dtype: object

In [5]:
df['sqft_basement'].value_counts

<bound method IndexOpsMixin.value_counts of 0          0.0
1        400.0
2          0.0
3        910.0
4          0.0
         ...  
21592      0.0
21593      0.0
21594      0.0
21595      0.0
21596      0.0
Name: sqft_basement, Length: 21597, dtype: object>

In [6]:
df[df['sqft_basement']== '?']['sqft_basement'].count()

454

In [7]:
## Replacing ? with zero,considering those to be houses without a basement

In [8]:
df['sqft_basement'] = df['sqft_basement'].replace('?',0)
df[df['sqft_basement']== 0 ]['sqft_basement'].count()

454

In [9]:
## Converting the column to a float
df['sqft_basement'] = df['sqft_basement'].astype(str).astype(float)
df['sqft_basement'].dtype

dtype('float64')

The date column is an object data type. Converting it to a datetime object would make it more useful to the model.

In [10]:
df['date'] = pd.to_datetime(df['date'])
df['date'].dtype

dtype('<M8[ns]')

In [11]:
## Taking a look at the grade and condition columns
df['grade'].unique()

array(['7 Average', '6 Low Average', '8 Good', '11 Excellent', '9 Better',
       '5 Fair', '10 Very Good', '12 Luxury', '4 Low', '3 Poor',
       '13 Mansion'], dtype=object)

In [12]:
df['condition'].unique()

array(['Average', 'Very Good', 'Good', 'Poor', 'Fair'], dtype=object)

In [13]:
## Changing the grade column to only remain with strings ie removing the numbers
df['grade'] = df['grade'].map(lambda x: x[1:])
df['grade']

0             Average
1             Average
2         Low Average
3             Average
4                Good
             ...     
21592            Good
21593            Good
21594         Average
21595            Good
21596         Average
Name: grade, Length: 21597, dtype: object

In [14]:
## Stripping the spaces
df['grade'] = df['grade'].apply(lambda x: x.rstrip())
df['grade']

0             Average
1             Average
2         Low Average
3             Average
4                Good
             ...     
21592            Good
21593            Good
21594         Average
21595            Good
21596         Average
Name: grade, Length: 21597, dtype: object

### Dropping irrelevant columns

From the documentation it can be known that some columns will be irrelevant to our project. This is because the have no predictive power,for instance the zipcode,lat and long columns. A house buyer or seller doesn't look at the latitudes or longitudes in order to consider buying a house (in the case of buyer) or prizing a house(in the case of a seller).The year renovated will not be considered relevant, provided the condition of the house is in our model there will be no use of the yr_renovated column. In addition it has missing values.

This is the list of columns to be dropped since they are not  considered relevant to our model = `['zipcode','lat','long','yr_renovated']`

In [21]:
df_r = df.drop(['zipcode','lat','long','yr_renovated'],axis = 1)
print(df.shape)
df_r.head()


(21597, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,Average,Average,1180,0.0,1955,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,Average,2170,400.0,1951,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,Average,Low Average,770,0.0,1933,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,Very Good,Average,1050,910.0,1965,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,Average,Good,1680,0.0,1987,1800,7503


### Dealing with missing values

In [22]:
df_r.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [28]:
## The waterfront and view columns have missing values both are categorical columns
## Taking a look at the view column
df_r['view'].value_counts()

NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

Houses with no view ie NONE occur  the most. We can replace the missing values with NONE

In [32]:
## Replacing the missing values in the view column with NONE
df_r['view'].fillna('NONE',inplace = True)
df_r['view'].value_counts()

NONE         19485
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [34]:
## Taking a look at the waterfront column
df_r['waterfront'].value_counts()

NO     19075
YES      146
Name: waterfront, dtype: int64

Dropping the rows with the missing values will not be a good idea since the number of missing values in the waterfront column is more than 2300. We will be removing more than 10% of our dataset. A rule of thumb is that we are not supposed to drop more than 5% of the entire dataset.Since houses with no waterfront occur the most it is more reasonable to replace the missing values with 'NO'. They will be considered as houses with no waterfront.

In [37]:
## Replacing the missing values in the waterfront column with NO
df_r['waterfront'].fillna('NO',inplace = True)
df_r['waterfront'].value_counts()

NO     21451
YES      146
Name: waterfront, dtype: int64

In [38]:
df_no_mv = df_r.copy()
df_no_mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21597 non-null  int64         
 1   date           21597 non-null  datetime64[ns]
 2   price          21597 non-null  float64       
 3   bedrooms       21597 non-null  int64         
 4   bathrooms      21597 non-null  float64       
 5   sqft_living    21597 non-null  int64         
 6   sqft_lot       21597 non-null  int64         
 7   floors         21597 non-null  float64       
 8   waterfront     21597 non-null  object        
 9   view           21597 non-null  object        
 10  condition      21597 non-null  object        
 11  grade          21597 non-null  object        
 12  sqft_above     21597 non-null  int64         
 13  sqft_basement  21597 non-null  float64       
 14  yr_built       21597 non-null  int64         
 15  sqft_living15  2159

### Dealing with duplicates

In order to identify duplicates we need to set the subset to the id and date columns. A house can be sold twice but not on the same date. Factors such as the condition/grade might have changed its price.

In [53]:
df_no_mv.duplicated(subset = ['id','date']).sum()

0

In [54]:
## The dataset has no missing values

### Exploring the distributions and dealing with outliers

Most often non normality in the variables is associated with non normality in the errors.We can ensure normality in our variables before modelling and still confirm afterwards.Outliers will also be delt with since they also affect the distributions of our variables.

In [ ]:
#